1. 프레임워크
    : Keras를 사용하였습니다. Tensorflow보다는 개발을 좀 더 편리하게 하기 위해서 Keras를 사용하였습니다.
2. 데이터 전처리
    : 이미지 파일로 제공되는 줄 알았으나, 그림을 그린 벡터형식으로 주어졌습니다. 이 벡터 형식을 이미지 파일로 변환하는 전처리 과정을 수행하였습니다. 벡터 형식은 CV2.line를 사용하여 그림을 그렸습니다.
    또한 데이터가 256x256 크기였으나, 각 Label 당 100개정도 밖에 학습을 못할 정도로 주어진 자원(RAM)이 부족하여, 크기를 64x64로 축소하였습니다.
3. 학습 모델
    : CNN의 기본 모델을 사용하여 학습을 하였습니다. Convolution 레이어는 2개를 추가하였습니다. BatchSize = 400, epoch = 20 을 설정하고 학습하였습니다.
4. 결과
    : loss = 2.57703574775247 , acc = 0.43591177463531494 의 결과가 나왔습니다. 학습을 하는 데에 사용되는 데이터의 양이 상대적으로 부족하여 학습이 덜 된것 같습니다. RAM을 좀 더 효율적으로 사용하는 방법을 추가적으로 생각해 보아야 할 것 같습니다.
    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 as cv # image lining
import matplotlib.pyplot as plt # image
import os
import ast # string to list
from sklearn import model_selection # Train : Test set
from sklearn.preprocessing import OneHotEncoder # OneHotEncoding

from keras.models import Sequential # Keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, pooling
from keras.utils import np_utils

plt.gray() #graycolor setting

데이터 확인

In [ ]:
df = pd.read_csv("../input/quickdraw-doodle-recognition/train_simplified/cat.csv")

In [ ]:
df

vector drawing

In [ ]:
def vector_to_img(vector):
    image = np.zeros((256,256), np.uint8)
    for line in vector:
        for i in range(len(line[0])-1):
            cv.line(image,(line[0][i],line[1][i]),(line[0][i+1],line[1][i+1]),color=255)
    
    temp_image= cv.resize(image, dsize=(64, 64), interpolation=cv.INTER_AREA)
    del image
    return temp_image

In [ ]:
df['drawing'][0]

In [ ]:
vector_img = ast.literal_eval(df['drawing'][0])
a=vector_to_img(vector_img)
plt.imshow(a)

전처리

X - img

Y - label

In [ ]:
X = []
Y = []
Label = []
Label_num = 0

In [ ]:
def File_Preprocessing(filename,X,Y,Label_num):
    df = pd.read_csv("../input/quickdraw-doodle-recognition/train_simplified/{}".format(filename))
    for i in range(500):
        img = vector_to_img(ast.literal_eval(df['drawing'][i])).reshape(64,64,1)
        X.append(img)
        Y.append([Label_num])
        del img
    del df


train_simplified에 대해 전처리 실행

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/quickdraw-doodle-recognition/train_simplified'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        df = pd.read_csv("../input/quickdraw-doodle-recognition/train_simplified/{}".format(filename))
        Label.append(df['word'][0])
        for i in range(500):
            img = vector_to_img(ast.literal_eval(df['drawing'][i])).reshape(64,64,1)
            X.append(img)
            Y.append([Label_num])
        Label_num = Label_num + 1
        
enc = OneHotEncoder()
enc.fit(Y)
Y = enc.transform(Y).toarray()

print("Fin")

In [ ]:
Label

In [ ]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(np.array(X), np.array(Y), test_size = 0.2)

In [ ]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

CNN 학습

In [ ]:
#normalization
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
model = Sequential()
model.add(Conv2D(32, 3, 3, activation='relu', input_shape=(64,64, 1)))
print(model.output_shape)

model.add(Conv2D(32, 3, 3, activation='relu'))
model.add(pooling.MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
print(model.output_shape)

model.add(Conv2D(32, 3, 3, activation='relu'))
model.add(pooling.MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
print(model.output_shape)

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(340, activation='softmax'))
print(model.output_shape)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, batch_size=400, epochs=20, verbose=1)

In [ ]:
loss,acc = model.evaluate(X_test, Y_test, verbose=0)
print("loss = {} , acc = {}".format(loss,acc))

In [ ]:
model.save('kmj.hdf5')

test_simplified 에 대해 전처리 실행

In [ ]:
test_simplified = pd.read_csv("../input/quickdraw-doodle-recognition/test_simplified.csv")

In [ ]:
test_simplified

In [ ]:
test_simplified.shape

In [ ]:
X = []

In [ ]:
for i in range(112199):
    img = vector_to_img(ast.literal_eval(df['drawing'][i])).reshape(64,64,1)
    X.append(img)
print("Fin")

In [ ]:
X=np.array(X)
X.shape

In [ ]:
plt.imshow(X[0].reshape(64,64))

test_simplified Prediction

In [ ]:
pred = model.predict(X, verbose=1)

In [ ]:
top_3 = np.argsort(-pred)[:, 0:3]

In [ ]:
top_3

In [ ]:
top_3_Label = []
for i in top_3:
    top_3_Label.append("{} {} {}".format(Label[i[0]],Label[i[1]],Label[i[2]]))
print("Fin")

Submission

In [ ]:
sample_submission = pd.read_csv("../input/quickdraw-doodle-recognition/sample_submission.csv",index_col=['key_id'])
sample_submission

In [ ]:
submission = sample_submission

In [ ]:
submission['word'] = top_3_Label

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv')
submission.head()